In [45]:
import random
import re
import librosa
import torch
import torchaudio
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import models, layers


In [46]:
import torch.nn as nn

In [47]:
model_path = 'C:\\Users\\adwik\\Downloads\\Chroma_scale_12_raga_92_f1_big_model_cross_valid.keras'

# Check if the file exists
if not os.path.exists(model_path):
    raise ValueError(f"File not found: {model_path}. Please ensure the file path is correct and accessible.")

# Load the model
model = tf.keras.models.load_model(model_path)

# Summarize the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 938, 12, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 938, 12, 64)       1664      
                                                                 
 batch_normalization (Batch  (None, 938, 12, 64)       256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 938, 12, 64)       102464    
                                                                 
 batch_normalization_1 (Bat  (None, 938, 12, 64)       256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 938, 6, 64)        0     

In [4]:
file_data = pd.read_csv('D:\\Selected_501\\Selected_501_files.csv')
print(file_data)

    Original File Name processed_raga_names Detected_Raaga1  \
0        12259-1-1.wav               Abhogi          Abhogi   
1         8213-1-1.wav               Abhogi          Abhogi   
2        13278-1-1.wav       Darbari_kanada  Darbari_kanada   
3        12257-1-1.wav        Abhogi_kanada   Abhogi_kanada   
4        13284-1-1.wav               Khamaj          Khamaj   
..                 ...                  ...             ...   
496      12066-1-2.wav                Kamod           KAMOD   
497      11792-1-1.wav          Majh_khamaj     MAJH KHAMAJ   
498       8181-1-1.wav        Mishra_khamaj   MISHRA-KHAMAJ   
499      12384-1-1.wav        Mishra_khamaj   MISHRA-KHAMAJ   
500      11791-1-1.wav        Mishra_tilang   MISHRA-TILANG   

    Final Selected File  Unnamed: 0       0               Archive No.  \
0                   NaN      3601.0  3741.0  IN-AIR-DSA-GT-M00-024541   
1                   NaN       918.0  1058.0  IN-AIR-DSA-GT-M00-022160   
2                   NaN 

In [5]:
def get_files_by_raga(raga_name):
    # Filter rows where 'processed_raga_names' matches the given raga_name
    filtered_files = file_data[file_data['processed_raga_names'] == raga_name]
    
    # Extract 'Original File Name' for the filtered rows
    original_file_names = filtered_files['Original File Name']
    
    return original_file_names

# Example usage:
raga_name = 'Abhogi'  # Replace with any raga name you want to search for
abhogi_file_names = get_files_by_raga(raga_name)

# Display the file names
abhogi_file_names

0      12259-1-1.wav
1       8213-1-1.wav
157    14994-1-1.wav
Name: Original File Name, dtype: object

In [6]:
all_raga = file_data['processed_raga_names'].unique()
print(all_raga)
print(len(all_raga))

['Abhogi' 'Darbari_kanada' 'Abhogi_kanada' 'Khamaj' 'Adana'
 'Devgiri_bilawal' 'Bhairav' 'Bhimpalasi' 'Alhaiya_bilawal'
 'Vrindavani_sarang' 'Kaushiki_kanada' 'Kirwani' 'Devgandhar' 'Swanandi'
 'Bageshri' 'Puriya' 'Tilak_kamod' 'Bhairavi' 'Bageshri_bahar'
 'Suha_malhar' 'Bageshri_kanada' 'Raysa_kanada' 'Bahar' 'Basant'
 'Basant_bahar' 'Jayant_malhar' 'Basant_kedar' 'Sohni' 'Shyam_kalyan'
 'Jaunpuri' 'Gujri_todi' 'Puriya_kalyan' 'Nand' 'Tilak_shyam' 'Chayanat'
 'Charukeshi' 'Bhatiyar' 'Kafi_kanada' 'Bhinna_shadaja' 'Bhopali' 'Bihag'
 'Des' 'Aheer_bhairav' 'Durga' 'Jog_kauns' 'Malgunji' 'Bihagda' 'Bihari'
 'Chandrakauns (Bhairavi_ang)' 'Chandrakauns' 'Shankara'
 'Chandrakauns (Bageshri_ang)' 'Shuddha_kalyan' 'Kafi_malhar'
 'Kans_gandhar' 'Pahadi' 'Mishra_khamaj' 'Malkauns' 'Darbari'
 'Nayaki_kanada' 'Hansdhwani' 'Jog' 'Basant_bhairav' 'Shivmat_bhairav'
 'Rageshree' 'Paraj' 'Gaud_malhar' 'Gaud_sarang' 'Nat_bhairav' 'Gavati'
 'Gorakh_kalyan' 'Jaijaiwanti' 'Hameer' 'Todi (ahiri)' 'Sawani_bi

In [7]:
from collections import Counter

raga_counts = Counter(file_data['processed_raga_names'])

# Print the count of each unique raga
print(raga_counts)

Counter({'Bhairavi': 15, 'Sohni': 15, 'Jog': 14, 'Khamaj': 13, 'Bageshri': 12, 'Bhopali': 12, 'Bihag': 12, 'Shuddha_kalyan': 12, 'Nayaki_kanada': 12, 'Malkauns': 11, 'Maru_bihaag': 11, 'Jog_kauns': 10, 'Yaman': 10, 'Puriya_kalyan': 9, 'Mishra_khamaj': 9, 'Kedar': 9, 'Des': 8, 'Chandrakauns': 8, 'Pahadi': 8, 'Kafi': 8, 'Darbari_kanada': 6, 'Chayanat': 6, 'Shankara': 6, 'Jaijaiwanti': 6, 'Kalavati': 6, 'Mian_malhar': 6, 'Adana': 5, 'Puriya': 5, 'Gorakh_kalyan': 5, 'Madhukauns': 5, 'Mishra_pilu': 5, 'Basant': 4, 'Jaunpuri': 4, 'Nand': 4, 'Aheer_bhairav': 4, 'Bihagda': 4, 'Rageshree': 4, 'Gaud_malhar': 4, 'Kamod': 4, 'Kaushik_dhwani': 4, 'Majh_khamaj': 4, 'Megh': 4, 'Surdasi_malhar': 4, 'Saraswati': 4, 'Abhogi': 3, 'Bageshri_kanada': 3, 'Basant_bahar': 3, 'Basant_kedar': 3, 'Shyam_kalyan': 3, 'Gujri_todi': 3, 'Kafi_kanada': 3, 'Bhinna_shadaja': 3, 'Durga': 3, 'Hansdhwani': 3, 'Gavati': 3, 'Hameer': 3, 'Sawani_bihag': 3, 'Kaunsi_kanada': 3, 'Janasammohini': 3, 'Jhinjhoti': 3, 'Mishra_bhaira

In [8]:
#using_raga = [raga for raga, count in raga_counts.items() if count >= 10]
#using_raga = ['Khamaj', 'Des', 'Kedar', 'Bhairavi', 'Sohni', 'Bihag', 'Shuddha_kalyan', 'Malkauns', 'Nayaki_kanada', 'Jog', 'Yaman', 'Maru_bihaag']
using_raga = ['Bhairavi', 'Bihag', 'Des', 'Jog', 'Kedar', 'Khamaj', 'Malkauns', 'Maru_bihaag', 'Nayaki_kanada', 'Shuddha_kalyan', 'Sohni', 'Yaman']
# Print the filtered ragas
# Bhopali , jog_kauns,
print(using_raga)
print(len(using_raga))
num_classes = len(using_raga)
# using_raga = ['Bhairavi', 'Sohni', 'Jog', 'Khamaj', 'Bageshri']
# num_classes = len(using_raga)

['Bhairavi', 'Bihag', 'Des', 'Jog', 'Kedar', 'Khamaj', 'Malkauns', 'Maru_bihaag', 'Nayaki_kanada', 'Shuddha_kalyan', 'Sohni', 'Yaman']
12


In [9]:
raga_files_dict = {}

# Iterate over each unique raga and get the corresponding original file names
for raga in using_raga:
    # Filter rows where 'processed_raga_names' matches the current raga
    raga_files = file_data[file_data['processed_raga_names'] == raga]['Original File Name'].tolist()
    # Store the result in the dictionary
    raga_files_dict[raga] = raga_files

# Display the dictionary
raga_files_dict

{'Bhairavi': ['14368-1-1.wav',
  '8291-1-1.wav',
  '15760-1-1.wav',
  '12528-1-1.wav',
  '8252-1-1.wav',
  '8201-1-1.wav',
  '11072-1-1.wav',
  '12312-1-1.wav',
  '14372-1-1.wav',
  '13902-1-1.wav',
  '14613-1-1.wav',
  '13742-1-1.wav',
  '15095-1-1.wav',
  '15535-1-1.wav',
  '14666-1-1.wav'],
 'Bihag': ['14515-1-1.wav',
  '13893-1-1.wav',
  '10742-1-1.wav',
  '11474-1-1.wav',
  '12081-1-1.wav',
  '14098-1-1.wav',
  '17469-1-1.wav',
  '11357-1-1.wav',
  '11812-1-1.wav',
  '8217-1-1.wav',
  '14867-1-1.wav',
  '12275-1-1.wav'],
 'Des': ['14817-1-1.wav',
  '15139-1-1.wav',
  '12295-1-1.wav',
  '12012-1-1.wav',
  '12037-1-1.wav',
  '11200-1-1.wav',
  '13163-1-1.wav',
  '14053-1-1.wav'],
 'Jog': ['8227-1-1.wav',
  '12847-1-1.wav',
  '10713-1-1.wav',
  '11074-1-1.wav',
  '12460-1-1.wav',
  '10884-1-1.wav',
  '11826-1-1.wav',
  '11687-1-1.wav',
  '11553-1-1.wav',
  '12924-1-1.wav',
  '12662-1-1.wav',
  '12514-1-1.wav',
  '13008-1-1.wav',
  '13726-1-1.wav'],
 'Kedar': ['12034-1-1.wav',
  '1203

In [10]:
def split_parent_files(train_ratio,test_ratio,validate_ratio):
    X_train_parent_file, X_test_parent_file, X_validate_parent_file, y_train_parent_file, y_test_parent_file, y_validate_parent_file = [],[],[],[],[],[]
    for raga in raga_files_dict:
        random.shuffle(raga_files_dict[raga])
        l = len(raga_files_dict[raga])
        test_num = round(l*test_ratio) if l*test_ratio>1 else 1
        validate_num = round(l*validate_ratio) if l*validate_ratio>1 else 1
        train_num = l - test_num - validate_num
        for i in range(test_num):
            X_test_parent_file.append(raga_files_dict[raga][i])
            y_test_parent_file.append(raga)
        for i in range(validate_num):
            X_validate_parent_file.append(raga_files_dict[raga][i+test_num])
            y_validate_parent_file.append(raga)
        for i in range(train_num):
            X_train_parent_file.append(raga_files_dict[raga][test_num+validate_num+i])
            y_train_parent_file.append(raga)
    return X_train_parent_file, X_test_parent_file, X_validate_parent_file, y_train_parent_file, y_test_parent_file, y_validate_parent_file
            

In [11]:
X_train_parent_file, X_test_parent_file, X_validate_parent_file, y_train_parent_file, y_test_parent_file, y_validate_parent_file = split_parent_files(0.8,0.1,0.1)

In [12]:
print(X_train_parent_file)

['13902-1-1.wav', '13742-1-1.wav', '14666-1-1.wav', '15760-1-1.wav', '8252-1-1.wav', '15095-1-1.wav', '15535-1-1.wav', '14613-1-1.wav', '14372-1-1.wav', '12312-1-1.wav', '14368-1-1.wav', '12275-1-1.wav', '14098-1-1.wav', '17469-1-1.wav', '11357-1-1.wav', '14867-1-1.wav', '13893-1-1.wav', '8217-1-1.wav', '12081-1-1.wav', '10742-1-1.wav', '11812-1-1.wav', '13163-1-1.wav', '12295-1-1.wav', '14817-1-1.wav', '12037-1-1.wav', '12012-1-1.wav', '11200-1-1.wav', '8227-1-1.wav', '12924-1-1.wav', '11826-1-1.wav', '13008-1-1.wav', '12662-1-1.wav', '11074-1-1.wav', '10713-1-1.wav', '12514-1-1.wav', '11553-1-1.wav', '11687-1-1.wav', '10884-1-1.wav', '13726-1-1.wav', '8290-1-1.wav', '12844-1-1.wav', '11145-1-1.wav', '14571-1-1.wav', '11088-1-1.wav', '12030-1-1.wav', '11819-1-1.wav', '13193-1-1.wav', '14977-1-1.wav', '12269-1-1.wav', '13284-1-1.wav', '8160-1-1.wav', '12519-1-1.wav', '12805-1-1.wav', '13069-1-1.wav', '12279-1-1.wav', '11958-1-1.wav', '8002-1-1.wav', '12184-1-1.wav', '12069-1-1.wav', '1

In [13]:
print(X_test_parent_file)

['8201-1-1.wav', '12528-1-1.wav', '14515-1-1.wav', '14053-1-1.wav', '12847-1-1.wav', '11933-1-1.wav', '14387-1-1.wav', '12188-1-1.wav', '10957-1-1.wav', '12438-1-1.wav', '8211-1-1.wav', '10824-1-1.wav', '13700-1-1.wav', '10916-1-1.wav']


In [14]:
print(X_validate_parent_file)

['11072-1-1.wav', '8291-1-1.wav', '11474-1-1.wav', '15139-1-1.wav', '12460-1-1.wav', '12034-1-1.wav', '11522-1-1.wav', '12537-1-1.wav', '8250-1-1.wav', '10807-1-1.wav', '12436-1-1.wav', '11217-1-1.wav', '13741-1-1.wav', '14573-1-1.wav']


In [15]:
print(y_train_parent_file)
print(y_test_parent_file)
print(y_validate_parent_file)

['Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bhairavi', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Bihag', 'Des', 'Des', 'Des', 'Des', 'Des', 'Des', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Jog', 'Kedar', 'Kedar', 'Kedar', 'Kedar', 'Kedar', 'Kedar', 'Kedar', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Khamaj', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Malkauns', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Maru_bihaag', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Nayaki_kanada', 'Shuddha_kalyan', 'Shuddha_kalyan', 'Shuddha_kalyan', 'Shuddha_ka

In [16]:
print(len(y_train_parent_file))
print(len(y_test_parent_file))
print(len(y_validate_parent_file))
print(len(X_train_parent_file))
print(len(X_test_parent_file))
print(len(X_validate_parent_file))

114
14
14
114
14
14


In [17]:
chunk_audio_data = pd.read_csv('D:\\Selected_501\\extract_correct_music_speech_timings.csv')
chunk_audio_data

,Original File Name,speech,music
0,12259-1-1.wav,"12259-1_1.wav,12259-1_2.wav","12259-1_4.wav,12259-1_5.wav,12259-1_6.wav,1225..."
1,8213-1-1.wav,8213-1_1.wav,"8213-1_3.wav,8213-1_4.wav,8213-1_5.wav,8213-1_..."
2,13278-1-1.wav,"13278-1_1.wav,13278-1_2.wav,13278-1_3.wav,1327...","13278-1_10.wav,13278-1_11.wav,13278-1_12.wav,1..."
3,12257-1-1.wav,"12257-1_1.wav,12257-1_2.wav","12257-1_4.wav,12257-1_5.wav,12257-1_6.wav,1225..."
4,14406-1-1.wav,"14406-1_1.wav,14406-1_2.wav","14406-1_4.wav,14406-1_5.wav,14406-1_6.wav,1440..."
...,...,...,...
496,12667-1-1.wav,NaN,"12667-1_1.wav,12667-1_2.wav,12667-1_3.wav,1266..."
497,12033-1-1.wav,"12033-1_1.wav,12033-1_2.wav,12033-1_3.wav,1203...","12033-1_7.wav,12033-1_8.wav,12033-1_9.wav,1203..."
498,10916-1-1.wav,"10916-1_1.wav,10916-1_2.wav,10916-1_3.wav,1091...","10916-1_8.wav,10916-1_9.wav,10916-1_10.wav,109..."
499,14661-1-1.wav,"14661-1_1.wav,14661-1_2.wav,14661-1_3.wav,1466...","14661-1_8.wav,14661-1_9.wav,14661-1_10.wav,146..."


In [18]:
master_tonic_csv = pd.read_excel('D:\\Selected_501\\master_tonic_501_v2.xlsx')
master_tonic_csv

,Unnamed: 0,Final Selected File,Audio_Number,Tonic,Remarks
0,0,NaN,12259-1-1.wav,C#,NaN
1,1,NaN,8213-1-1.wav,C#,NaN
2,2,NaN,13278-1-1.wav,G#,NaN
3,3,NaN,12257-1-1.wav,C#,NaN
4,4,NaN,14406-1-1.wav,C#,less audible
...,...,...,...,...,...
496,496,YAMAN--2606.wav,12667-1-1.wav,C#,NaN
497,497,YAMAN--2710.wav,12033-1-1.wav,A#,NaN
498,498,YAMAN--3016.wav,10916-1-1.wav,B,NaN
499,499,SHUDHA_KALYAN--1681.wav,14661-1-1.wav,C#,2 different singers singing


In [19]:
import pandas as pd
import numpy as np

# Define the mapping for the musical notes
mapping_notes = {'G': 0, 'G#': 1, 'A': 2, 'A#': 3, 'B': 4, 'C': 5, 'C#': 6, 'D': 7, 'D#': 8, 'E': 9, 'F': 10, 'F#': 11}
# Map the 'Tonic' column
for i in range(len(master_tonic_csv)):
    val = master_tonic_csv.loc[i, 'Tonic']
    if pd.isna(val):
        master_tonic_csv.loc[i, 'Tonic'] = 0
    elif val in mapping_notes:
        master_tonic_csv.loc[i, 'Tonic'] = mapping_notes[val]
    else:
        print(f"Unexpected value '{val}' in 'Tonic' column at index {i}")
        master_tonic_csv.loc[i, 'Tonic'] = -1  

# Save the updated DataFrame back to a CSV file
master_tonic_csv.to_csv('path_to_your_updated_csv_file.csv', index=False)

In [20]:
tonic = []

# Iterate through each row in the chunk_audio_data DataFrame
for i in range(len(chunk_audio_data)):
    # Get the 'Original File Name' for the current row
    name = chunk_audio_data['Original File Name'][i]
    
    # Find the index of the row in master_tonic_csv where 'Audio_Number' matches 'name'
    row = master_tonic_csv.index[master_tonic_csv['Audio_Number'] == name].tolist()
    
    # Check if a matching row was found
    if row:
        # Extract the tonic value from the matching row and append it to the tonic list
        tonic_value = master_tonic_csv['Tonic'].iloc[row[0]]
        tonic.append(tonic_value)
    else:
        # Append 0 to the tonic list if no matching row is found
        tonic.append(0)

# Print the list of tonic values
print(tonic)

[6, 6, 1, 6, 6, 1, 6, 6, 1, 6, 6, 6, 6, 8, 6, 7, 8, 6, 7, 0, 8, 2, 8, 0, 7, 8, 1, 8, 6, 8, 6, 7, 8, 5, 3, 7, 1, 6, 9, 6, 7, 8, 1, 2, 6, 8, 3, 2, 2, 9, 1, 2, 6, 6, 6, 2, 1, 8, 1, 6, 6, 7, 6, 9, 8, 6, 9, 7, 2, 2, 3, 3, 6, 6, 1, 10, 2, 9, 6, 6, 6, 3, 6, 3, 8, 5, 8, 6, 7, 6, 7, 6, 7, 1, 8, 1, 9, 8, 7, 9, 1, 7, 2, 2, 1, 1, 6, 6, 2, 2, 3, 6, 8, 6, 8, 8, 5, 3, 6, 6, 1, 7, 1, 7, 8, 3, 3, 1, 7, 8, 1, 6, 6, 8, 8, 5, 6, 8, 6, 7, 7, 2, 6, 3, 3, 8, 7, 7, 7, 6, 6, 6, 6, 0, 6, 6, 1, 3, 8, 0, 7, 3, 8, 6, 6, 3, 6, 7, 7, 1, 2, 6, 8, 8, 7, 1, 6, 6, 7, 1, 6, 7, 9, 8, 1, 7, 1, 8, 3, 1, 8, 8, 5, 3, 8, 6, 2, 8, 1, 7, 6, 6, 6, 2, 8, 6, 4, 6, 7, 6, 7, 6, 2, 7, 6, 7, 1, 9, 3, 8, 1, 8, 1, 1, 8, 3, 2, 1, 1, 6, 1, 3, 6, 2, 3, 7, 8, 3, 8, 6, 6, 1, 2, 2, 6, 7, 6, 3, 5, 6, 1, 7, 1, 3, 7, 7, 8, 2, 1, 1, 6, 6, 6, 6, 6, 1, 0, 7, 6, 6, 1, 2, 6, 6, 6, 9, 5, 6, 7, 1, 3, 8, 8, 7, 1, 1, 7, 8, 3, 7, 2, 2, 6, 8, 3, 8, 7, 3, 3, 6, 6, 6, 8, 0, 6, 7, 1, 8, 8, 7, 8, 3, 3, 8, 6, 9, 6, 8, 6, 6, 7, 6, 5, 6, 7, 6, 9, 9, 8, 9, 7, 8, 7,

In [21]:
chunk_audio_data['Tonic'] = tonic

In [22]:
chunk_audio_data

,Original File Name,speech,music,Tonic
0,12259-1-1.wav,"12259-1_1.wav,12259-1_2.wav","12259-1_4.wav,12259-1_5.wav,12259-1_6.wav,1225...",6
1,8213-1-1.wav,8213-1_1.wav,"8213-1_3.wav,8213-1_4.wav,8213-1_5.wav,8213-1_...",6
2,13278-1-1.wav,"13278-1_1.wav,13278-1_2.wav,13278-1_3.wav,1327...","13278-1_10.wav,13278-1_11.wav,13278-1_12.wav,1...",1
3,12257-1-1.wav,"12257-1_1.wav,12257-1_2.wav","12257-1_4.wav,12257-1_5.wav,12257-1_6.wav,1225...",6
4,14406-1-1.wav,"14406-1_1.wav,14406-1_2.wav","14406-1_4.wav,14406-1_5.wav,14406-1_6.wav,1440...",6
...,...,...,...,...
496,12667-1-1.wav,NaN,"12667-1_1.wav,12667-1_2.wav,12667-1_3.wav,1266...",6
497,12033-1-1.wav,"12033-1_1.wav,12033-1_2.wav,12033-1_3.wav,1203...","12033-1_7.wav,12033-1_8.wav,12033-1_9.wav,1203...",3
498,10916-1-1.wav,"10916-1_1.wav,10916-1_2.wav,10916-1_3.wav,1091...","10916-1_8.wav,10916-1_9.wav,10916-1_10.wav,109...",4
499,14661-1-1.wav,"14661-1_1.wav,14661-1_2.wav,14661-1_3.wav,1466...","14661-1_8.wav,14661-1_9.wav,14661-1_10.wav,146...",6


In [23]:
i = 61
print(chunk_audio_data['Original File Name'][i], chunk_audio_data['Tonic'][i])

8201-1-1.wav 7


In [24]:
# def form_chunk_data(X, y):
#     X_data, y_data = [], []
    
#     for i in range(len(X)):
#         # Find rows in chunk_audio_data where Original_file_data matches the current audio file
#         matching_row = chunk_audio_data[chunk_audio_data['Original File Name'] == X[i]].index
        
#         # Iterate over the matching rows and append each element in the 'music' column to X_data
#         for elements in chunk_audio_data['music'][matching_row]:
#             for element in elements.split(','):
#                 X_data.append(element)
#                 y_data.append(y[i])
    
#     return X_data, y_data

def form_chunk_data(X, y):
    X_data, y_data, tonic = [], [], []
    
    for i in range(len(X)):
        # Find rows in chunk_audio_data where Original_file_data matches the current audio file
        matching_row = chunk_audio_data[chunk_audio_data['Original File Name'] == X[i]].index
        
        # Iterate over the matching rows and append each element in the 'music' column to X_data
        for elements in chunk_audio_data['music'][matching_row]:
            for element in elements.split(','):
                X_data.append(element)
                y_data.append(y[i])
                tonic.append(chunk_audio_data['Tonic'].iloc[matching_row].values[0])

    
    return X_data, y_data, tonic


In [25]:
X_test_chunk,y_test_label, tonic_test = form_chunk_data(X_test_parent_file, y_test_parent_file)
X_train_chunk,y_train_label, tonic_train = form_chunk_data(X_train_parent_file, y_train_parent_file)
X_validate_chunk,y_validate_label, tonic_validate = form_chunk_data(X_validate_parent_file, y_validate_parent_file)
print(X_test_chunk)
print(y_test_label)
print(tonic_test)


['8201-1_3.wav', '8201-1_4.wav', '8201-1_5.wav', '8201-1_6.wav', '8201-1_7.wav', '8201-1_8.wav', '8201-1_9.wav', '8201-1_10.wav', '8201-1_11.wav', '12528-1_3.wav', '12528-1_4.wav', '12528-1_5.wav', '12528-1_6.wav', '12528-1_7.wav', '12528-1_8.wav', '12528-1_9.wav', '12528-1_10.wav', '12528-1_11.wav', '12528-1_12.wav', '12528-1_13.wav', '12528-1_14.wav', '12528-1_15.wav', '12528-1_16.wav', '12528-1_17.wav', '12528-1_18.wav', '12528-1_19.wav', '12528-1_20.wav', '14515-1_8.wav', '14515-1_9.wav', '14515-1_10.wav', '14515-1_11.wav', '14515-1_12.wav', '14515-1_13.wav', '14515-1_14.wav', '14515-1_15.wav', '14515-1_16.wav', '14515-1_17.wav', '14515-1_18.wav', '14515-1_19.wav', '14515-1_20.wav', '14515-1_21.wav', '14515-1_22.wav', '14515-1_23.wav', '14515-1_24.wav', '14515-1_25.wav', '14515-1_26.wav', '14515-1_27.wav', '14515-1_28.wav', '14515-1_29.wav', '14515-1_30.wav', '14515-1_31.wav', '14515-1_32.wav', '14515-1_33.wav', '14515-1_34.wav', '14515-1_35.wav', '14515-1_36.wav', '14515-1_37.wav'

In [26]:
def shift_chromogram(ch, tonic):
    num_rows = ch.shape[0] 
    shift_amount = tonic
    shifted_ch = np.roll(ch, shift_amount, axis=0)
    return shifted_ch

In [27]:
ch = np.random.rand(12, 1292)  # Example random chromogram
tonic = 2
shifted_ch = shift_chromogram(ch, tonic)

# Print the first few rows to verify the shift
print("Original chromogram:")
print(ch[:, :5])  # Print the first 3 rows and 5 columns
print("\nShifted chromogram:")
print(shifted_ch[:, :5])  # Print the first 3 rows and 5 columns of the shifted matrix

Original chromogram:
[[0.04474451 0.95311814 0.97968456 0.29474343 0.43003674]
 [0.3117533  0.95111207 0.02830105 0.03947134 0.83527067]
 [0.86075783 0.09462407 0.54695856 0.73026045 0.92850859]
 [0.84487112 0.75991088 0.03098692 0.34787909 0.84259465]
 [0.00289512 0.5566132  0.34535088 0.97634624 0.24976209]
 [0.05319388 0.53184427 0.13328991 0.25589568 0.87294223]
 [0.25395851 0.32276731 0.08599558 0.92760923 0.76161558]
 [0.56552356 0.66270591 0.20296777 0.6879569  0.02974236]
 [0.49164876 0.68547587 0.71213433 0.76166328 0.13258367]
 [0.879047   0.17401789 0.61745197 0.79823018 0.55393242]
 [0.6510346  0.09205637 0.83787984 0.57472829 0.35297536]
 [0.21088666 0.79313216 0.60058088 0.72859871 0.56942331]]

Shifted chromogram:
[[0.6510346  0.09205637 0.83787984 0.57472829 0.35297536]
 [0.21088666 0.79313216 0.60058088 0.72859871 0.56942331]
 [0.04474451 0.95311814 0.97968456 0.29474343 0.43003674]
 [0.3117533  0.95111207 0.02830105 0.03947134 0.83527067]
 [0.86075783 0.09462407 0.546

In [28]:
y,sr = librosa.load('D:\\Selected_501\\vocal_501_audio_chunks\\20979-1_37.wav', sr = None)
print(sr)
chromagram = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
print(chromagram.shape)

16000
(12, 938)


In [29]:
path = 'D:\\Selected_501\\vocal_501_audio_chunks'
def form_dataset(X,y_,tonic):
    X_wav,y_wav = [],[]
    for i in range(len(X)):
        path_wav = os.path.join(path, X[i])
        if os.path.exists(path_wav):
            y,sr = librosa.load(path_wav, sr = None)
            ch = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=512)
            if(ch.shape[1] == 938):
                ch_shifted = shift_chromogram(ch,tonic)
                z=ch_shifted.T
                #print(z.shape)
                chromagram=z.reshape(938,12,1)
                y_wav.append(y_[i])
                X_wav.append(chromagram)

    return X_wav , y_wav   

In [30]:
len(y_test_label)

591

In [31]:
X_test_ch, y_test_label_clean = form_dataset(X_test_chunk,y_test_label,tonic_test )
print(X_test_ch)

[array([[[0.44502178],
        [0.1951969 ],
        [0.4943511 ],
        ...,
        [0.35864356],
        [0.09558731],
        [0.3069077 ]],

       [[0.20098644],
        [0.19208091],
        [0.8842957 ],
        ...,
        [0.18372318],
        [0.09405905],
        [0.27517796]],

       [[0.04733099],
        [0.17531502],
        [1.        ],
        ...,
        [0.04980438],
        [0.10645068],
        [0.3076721 ]],

       ...,

       [[0.74221814],
        [0.5805239 ],
        [1.        ],
        ...,
        [0.59218246],
        [0.35436693],
        [0.83774257]],

       [[0.40726587],
        [0.3281853 ],
        [0.481934  ],
        ...,
        [0.5196292 ],
        [0.3680253 ],
        [0.40440166]],

       [[0.14543816],
        [0.14745551],
        [0.26889437],
        ...,
        [0.39330617],
        [0.2603127 ],
        [0.23749447]]], dtype=float32), array([[[0.07624685],
        [0.07751281],
        [0.46202385],
        ...,
        [

In [32]:
y_test_label_clean

['Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',
 'Bihag',

In [33]:
X_test_ch = np.array(X_test_ch)

In [34]:
X_test_ch.shape

(587, 938, 12, 1)

In [35]:
X_train_ch, y_train_label_clean = form_dataset(X_train_chunk, y_train_label,tonic_train)
X_validate_ch, y_validate_label_clean = form_dataset(X_validate_chunk, y_validate_label, tonic_validate)

C:\Users\adwik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [36]:
X_train_ch = np.array(X_train_ch)
X_validate_ch = np.array(X_validate_ch)

In [37]:
print(X_train_ch.shape)
print(X_validate_ch.shape)

(4578, 938, 12, 1)
(541, 938, 12, 1)


In [38]:
label_encoder = LabelEncoder()

all_labels = y_test_label_clean + y_train_label_clean + y_validate_label_clean
label_encoder.fit(all_labels)

y_test = label_encoder.transform(y_test_label_clean)
y_train = label_encoder.transform(y_train_label_clean)
y_validate = label_encoder.transform(y_validate_label_clean)

print("Encoded y_test_label:", y_test)
print("Encoded y_train_label:", y_train)
print("Encoded y_validate_label:", y_validate)

Encoded y_test_label: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  

In [39]:
print("Decoded y_test_label:", label_encoder.inverse_transform(y_test))
print("Decoded y_train_label:", label_encoder.inverse_transform(y_train))
print("Decoded y_validate_label:", label_encoder.inverse_transform(y_validate))

Decoded y_test_label: ['Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi'
 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi'
 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi'
 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bhairavi'
 'Bhairavi' 'Bhairavi' 'Bhairavi' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag'
 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bihag' 'Bi

In [40]:
y_pred_probs[2]

NameError: name 'y_pred_probs' is not defined

In [41]:
y_test[2]

0

In [42]:
from sklearn.metrics import classification_report
import numpy as np


# Predict the classes for the test set
y_pred_probs = model.predict(X_test_ch)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate the classification report
class_names = using_raga  # Assuming using_raga is a list of class names
print(classification_report(y_test, y_pred, target_names=class_names))

19/19 [==============================] - 25s 1s/step
                precision    recall  f1-score   support

      Bhairavi       0.07      0.30      0.12        27
         Bihag       0.00      0.00      0.00        96
           Des       0.00      0.00      0.00        14
           Jog       0.03      0.04      0.04        26
         Kedar       0.00      0.00      0.00        75
        Khamaj       0.00      0.00      0.00         4
      Malkauns       0.80      1.00      0.89        60
   Maru_bihaag       0.00      0.00      0.00        71
 Nayaki_kanada       0.00      0.00      0.00        15
Shuddha_kalyan       0.00      0.00      0.00        52
         Sohni       0.00      0.00      0.00        52
         Yaman       0.05      0.15      0.08        95

      accuracy                           0.14       587
     macro avg       0.08      0.12      0.09       587
  weighted avg       0.10      0.14      0.11       587



C:\Users\adwik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\adwik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\adwik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [43]:
# Predict the classes for the test set
y_pred_probs = model.predict(X_train_ch)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate the classification report
class_names = using_raga  # Assuming using_raga is a list of class names
print(classification_report(y_train, y_pred, target_names=class_names))

144/144 [==============================] - 225s 2s/step
                precision    recall  f1-score   support

      Bhairavi       0.03      0.23      0.06       155
         Bihag       0.79      0.38      0.51       519
           Des       0.10      0.08      0.09       113
           Jog       0.79      0.36      0.50       617
         Kedar       0.83      0.65      0.73       291
        Khamaj       0.13      0.15      0.14       172
      Malkauns       0.09      0.03      0.05       454
   Maru_bihaag       0.72      0.17      0.28       495
 Nayaki_kanada       0.25      0.21      0.23       444
Shuddha_kalyan       0.36      0.29      0.32       580
         Sohni       0.16      0.29      0.21       238
         Yaman       0.31      0.51      0.38       500

      accuracy                           0.30      4578
     macro avg       0.38      0.28      0.29      4578
  weighted avg       0.45      0.30      0.33      4578



In [44]:
# Predict the classes for the test set
y_pred_probs = model.predict(X_validate_ch)
y_pred = np.argmax(y_pred_probs, axis=1)

# Generate the classification report
class_names = using_raga  # Assuming using_raga is a list of class names
print(classification_report(y_validate, y_pred, target_names=class_names))

17/17 [==============================] - 38s 2s/step
                precision    recall  f1-score   support

      Bhairavi       0.00      0.00      0.00        38
         Bihag       0.75      0.95      0.84        40
           Des       0.00      0.00      0.00        23
           Jog       0.78      1.00      0.88        42
         Kedar       0.00      0.00      0.00        23
        Khamaj       0.33      0.08      0.12        26
      Malkauns       0.08      0.02      0.03        46
   Maru_bihaag       0.00      0.00      0.00        67
 Nayaki_kanada       0.33      0.03      0.05        40
Shuddha_kalyan       0.00      0.00      0.00        89
         Sohni       0.16      0.20      0.18        41
         Yaman       0.00      0.00      0.00        66

      accuracy                           0.17       541
     macro avg       0.20      0.19      0.17       541
  weighted avg       0.17      0.17      0.16       541



In [21]:
pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\adwik\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [72]:
from transformers import Wav2Vec2FeatureExtractor, AutoModel
import torch
import torchaudio.transforms as T
import librosa

# Load model and processor
model = AutoModel.from_pretrained("m-a-p/MERT-v0-public", trust_remote_code=True)
processor = Wav2Vec2FeatureExtractor.from_pretrained("m-a-p/MERT-v0-public", trust_remote_code=True)


Some weights of the model checkpoint at m-a-p/MERT-v0-public were not used when initializing MERTModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing MERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MERTModel were not initialized from the model checkpoint at m-a-p/MERT-v0-public and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

In [ ]:
# def form_dataset(X,y):
#     X_wav,y_wav = [],[]
#     path = 'D:\\Selected_50\\vocal_501_audio_chunks'
#     for i in range(100):
#         path_wav = path + X[i]
#         if os.path.exists(path_wav):
#             data, sample_rate = librosa.load(path_wav, sr = 16000)
#             if(data.shape[0] == 480000):
#                 inputs = processor(data, sampling_rate= sample_rate, return_tensors="pt")
#                 with torch.no_grad():
#                     outputs = model(**inputs, output_hidden_states=True)

#                 all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()
#                 print(all_layer_hidden_states.shape)  # [13, Time steps, 768 feature_dim]

#                 time_reduced_hidden_states = all_layer_hidden_states.mean(-2)
#                 print(time_reduced_hidden_states.shape)  # [13, 768]

#                 aggregator = nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1)
#                 weighted_avg_hidden_states = aggregator(time_reduced_hidden_states.unsqueeze(0)).squeeze()
#                 print(weighted_avg_hidden_states.shape)  # [768]
#                 y_wav.append(weighted_avg_hidden_states)
#                 X_wav.append(data)
#     return X_wav,y_wav

def form_dataset(X, y):
    X_wav, y_wav = [], []
    path = 'D:\\Selected_501\\vocal_501_audio_chunks'
    
    for i in range(100):
        path_wav = os.path.join(path, X[i])  # Construct full path to audio file
        if os.path.exists(path_wav):
            print(f"Processing file: {path_wav}")
            data, sample_rate = librosa.load(path_wav, sr=16000)
            
            if data.shape[0] == 480000:
                inputs = processor(data, sampling_rate=sample_rate, return_tensors="pt")
                
                with torch.no_grad():
                    outputs = model(**inputs, output_hidden_states=True)

                all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()
                print(all_layer_hidden_states.shape)  # [13, Time steps, 768 feature_dim]

                time_reduced_hidden_states = all_layer_hidden_states.mean(-2)
                print(time_reduced_hidden_states.shape)  # [13, 768]

                aggregator = nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1)
                weighted_avg_hidden_states = aggregator(time_reduced_hidden_states.unsqueeze(0)).squeeze()
                print(weighted_avg_hidden_states.shape)  # [768]
                
                y_wav.append(weighted_avg_hidden_states)
                X_wav.append(data)
            else:
                print(f"Ignoring file {path_wav}: Incorrect shape {data.shape[0]} != 661500")
        else:
            print(f"File not found: {path_wav}")

    return X_wav, y_wav


In [34]:
X_test, y_test_label_clean = [],[]
X_test,y_test_label_clean = form_dataset(X_test_chunk,y_test_label)

Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_3.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_4.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_5.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_6.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_7.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_8.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\vocal_501_audio_chunks\14666-1_9.wav
torch.Size([13, 1499, 768])
torch.Size([13, 768])
torch.Size([768])
Processing file: D:\Selected_501\v

KeyboardInterrupt: 

In [23]:
# Load and process audio
audio_path = 'D:\\Selected_501\\vocal_501_audio_chunks\\7122-1_3.wav'
y, sr = librosa.load(audio_path, sr=16000)

# Check if resampling is necessary
resample_rate = processor.sampling_rate
if sr != resample_rate:
    print(f'Resampling audio from {sr} to {resample_rate}')
    resampler = T.Resample(sr, resample_rate)
    y = resampler(torch.tensor(y)).numpy()

# Process audio using processor
inputs = processor(y, sampling_rate=resample_rate, return_tensors="pt")

# Feed input to the model
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

# Access and print the shape of the hidden states
hidden_states = outputs.hidden_states
print("Number of hidden states:", len(hidden_states))  # Should be 13 layers for Wav2Vec2
print("Shape of each hidden state layer:")
for i, layer in enumerate(hidden_states):
    print(f"Layer {i}: {layer.shape}")

# Post-processing of model output
all_layer_hidden_states = torch.stack(hidden_states).squeeze()
time_reduced_hidden_states = all_layer_hidden_states.mean(-2)

aggregator = nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1)
weighted_avg_hidden_states = aggregator(time_reduced_hidden_states.unsqueeze(0)).squeeze()

print("Shape of weighted average hidden states:", weighted_avg_hidden_states.shape)

Number of hidden states: 13
Shape of each hidden state layer:
Layer 0: torch.Size([1, 1499, 768])
Layer 1: torch.Size([1, 1499, 768])
Layer 2: torch.Size([1, 1499, 768])
Layer 3: torch.Size([1, 1499, 768])
Layer 4: torch.Size([1, 1499, 768])
Layer 5: torch.Size([1, 1499, 768])
Layer 6: torch.Size([1, 1499, 768])
Layer 7: torch.Size([1, 1499, 768])
Layer 8: torch.Size([1, 1499, 768])
Layer 9: torch.Size([1, 1499, 768])
Layer 10: torch.Size([1, 1499, 768])
Layer 11: torch.Size([1, 1499, 768])
Layer 12: torch.Size([1, 1499, 768])
Shape of weighted average hidden states: torch.Size([768])


In [29]:
print(weighted_avg_hidden_states.detach().numpy())

[ 5.44511713e-02  1.35319093e-02  1.02068745e-01 -1.93776214e-03
  8.48326162e-02  3.41747366e-02  2.27740947e-02  3.43693048e-02
  7.98876360e-02  7.26913139e-02  3.91413346e-02  1.13640830e-01
  5.48096262e-02  6.03024811e-02  1.14025213e-01  1.03414468e-01
  8.30825269e-02  2.06121057e-02  1.39281889e-02  5.85442036e-02
  1.72749143e-02  7.96685442e-02  3.34364828e-03 -3.05139069e-02
  9.48893931e-03 -2.40392182e-02 -4.85160649e-02  1.31682649e-01
  7.33902752e-02  9.51518267e-02  3.99515741e-02  9.60670561e-02
 -4.91922200e-02 -2.54700854e-02  1.01849278e-02  3.85039449e-02
 -8.00486561e-03  9.55312252e-02  8.46824273e-02  1.41723352e-02
  1.04607761e-01  8.53325799e-02  9.29545984e-02  3.63465026e-02
 -3.61597911e-02  4.27942397e-03  6.99452087e-02  1.24999359e-01
  2.58078519e-02  7.25948587e-02  2.78994739e-02  1.15330175e-01
 -1.97563116e-02  1.07943922e-01  7.26659596e-02  8.17870125e-02
  1.18774697e-01  2.45694313e-02  6.19418621e-02  7.05306455e-02
  1.07990041e-01  1.24410

In [73]:
def form_dataset(X, y, folder_name):
    X_wav, y_wav = [], []
    path = 'D:\\Selected_501\\vocal_501_audio_chunks'
    base_output_path = os.path.join('output', folder_name)  # Base output directory
    X_output_path = os.path.join(base_output_path, 'X_' + folder_name)  # Directory for X files
    # y_output_path = os.path.join(base_output_path, 'y_' + folder_name)  # Directory for y files

    os.makedirs(X_output_path, exist_ok=True)
    # os.makedirs(y_output_path, exist_ok=True)

    for i in range(len(X)):
        path_wav = os.path.join(path, X[i])
        if os.path.exists(path_wav):
            data, sample_rate = librosa.load(path_wav, sr=16000)
            if data.shape[0] == 480000:
                inputs = processor(data, sampling_rate=sample_rate, return_tensors="pt")
                with torch.no_grad():
                    outputs = model(**inputs, output_hidden_states=True)

                all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()
                time_reduced_hidden_states = all_layer_hidden_states.mean(-2)
                aggregator = nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1)
                weighted_avg_hidden_states = aggregator(time_reduced_hidden_states.unsqueeze(0)).squeeze()

                X_wav.append(weighted_avg_hidden_states)
                y_wav.append(y[i])
                file_tag = X[i].replace(".wav", "")
                # Save X_wav and y_wav to files
                X_output_file = os.path.join(X_output_path, f"X_MERT{file_tag}.npy")
                # y_output_file = os.path.join(y_output_path, f"y_{i}.pt")
                # torch.save(weighted_avg_hidden_states, X_output_file)
                np.save(X_output_file, weighted_avg_hidden_states.detach().numpy())
    return y_wav

In [74]:
y_test_label_clean = form_dataset(X_test_chunk, y_test_label, 'test_13')
y_train_label_clean = form_dataset(X_train_chunk, y_train_label, 'train_13')
y_validate_label_clean = form_dataset(X_validate_chunk, y_validate_label, 'validate_13')

In [42]:
y_test_label_clean

['Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Bhairavi',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Sohni',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog',
 'Jog'

In [45]:
y_output_test = os.path.join('D:\\Selected_501\\output\\test', 'y_test13')
os.makedirs(y_output_test, exist_ok=True)
file_path = os.path.join(y_output_test, 'y_test.txt')
with open(file_path ,'w') as file:
    for item in y_test_label_clean:
        file.write(item + '\n')


In [46]:
y_output_train = os.path.join('D:\\Selected_501\\output\\train', 'y_train13')
os.makedirs(y_output_train, exist_ok=True)
file_path = os.path.join(y_output_train, 'y_train.txt')
with open(file_path ,'w') as file:
    for item in y_train_label_clean:
        file.write(item + '\n')


In [48]:
y_output_validate = os.path.join('D:\\Selected_501\\output\\validate', 'y_validate13')
os.makedirs(y_output_validate, exist_ok=True)
file_path = os.path.join(y_output_validate, 'y_validate.txt')
with open(file_path ,'w') as file:
    for item in y_validate_label_clean:
        file.write(item + '\n')


In [ ]:

# class SelfAttention(nn.Module):
#     def __init__(self, embed_size, heads):
#         super(SelfAttention, self).__init__()
#         self.embed_size = embed_size #size of embeddings generated
#         self.heads = heads #
#         self.head_dim = embed_size // heads

#         assert (self.head_dim * heads == embed_size), "Embedding size is not completely divisible by number of heads you want to create"

#         self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
#         self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
#         self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
#         self.fc_out = nn.Linear(self.heads*self.head_dim, self.embed_size)

#         #the above linear layers is just a mapping between 2 spaces
#         #also self.heads*self.head_dim, self.embed_size, this is just to remind that both of these are equal

#         def forward(self, values, keys, queries, mask):
#             N = queries.shape[0] # how many examples do we send at a time
#             value_len, key_len, querie_len = values.shape[1], queries.shape[1], keys.shape[1]

#             #split embeddings into heads pieces
#             #assume that for our work there would be a vector of dimension 64
#             # V,Q,K all are of shape 2, 10, 64 that is 1 sequence contains 10 vectors with shape 64
#             #but in audio we have a dataset of the form (N,64), that is every input has a single vector , so w would need to make changes accordingly
#             values = values.reshape(N, value_len, self.heads, self.head_dim)
#             keys = keys.reshape(N, key_len, self.heads, self.head_dim)
#             queries = queries.reshape(N, querie_len, self.heads, self.dim)

#             energy = torch.einsum("nqhd, nkhd -> nhqk", [queries, keys])
#             if mask is not None:
#                 energy = energy.masked_fill(mask = 0, float("-1e20"))

#             attention = torch.softmax(energy / (self.embed_size ** 0.5), dim = 3)